In [1]:
import numpy as np
import do_mpc
from casadi import *

# Dynamic Model

This function defines the mathematical model for the toggle switch in the form of differential equations (ODE).

Defining $mRNA\_LacI$ the concentration of the mRNA associated with the _lacI gene_, $mRNA\_TetR$ the concentration of the mRNA associated with the _TetR gene_, $LacI$ the concentration of _LacI_ and $TetR$ the concentration of _TetR_, the system can be described via the following set of ODEs:

\begin{equation*}
    \begin{split}
        &\dot{mRNA_LacI} = k^{m_0}_L + k^m_L \frac{1}{1 + \left(\frac{TetR}{\theta_{TetR}} \frac{1}{1 + \left(\frac{v_1}{\theta_{aTc}}\right)^{\eta_{aTc}}}\right)^{\eta_{TetR}}} - g^m_L mRNA\_LacI\\
        &\dot{mRNA_TetR} = k^{m_0}_T + k^m_T \frac{1}{1 + \left(\frac{LacI}{\theta_{LacI}} \frac{1}{1 + \left(\frac{v_2}{\theta_{IPTG}}\right)^{\eta_{IPTG}}}\right)^{\eta_{LacI}}} - g^m_T mRNA\_TetR\\
        &\dot{LacI} = k^p_L mRNA\_LacI - g^p_L LacI\\
        &\dot{TetR} = k^p_T mRNA\_TetR - g^p_T TetR\\ \\
        &\dot{v_1} = \begin{cases} k^{in}_{aTc}(aTc - v_1) \quad if \quad  aTc > v_1\\ k^{out}_{aTc}(aTc - v_1) \quad if aTc \leq v_1 \end{cases}\\
        &\dot{v_2} = \begin{cases} k^{in}_{IPTG}(IPTG - v_2) \quad if \quad IPTG > v_2\\ k^{out}_{IPTG}(IPTG - v_2) \quad if IPTG \leq v_2 \end{cases}
    \end{split}
\end{equation*}

whre $aTc$ and $IPTG$ are extra-cellular concentrations of aTc and IPTg, respectively, and $v_1$ and $v_2$ are intra-cellular concentrations of aTc and IPTg, respectively.

The model parameters are not defined as they need to be estimated.

In [2]:
def template_model(stochasticity=False):

    model = do_mpc.model.Model(model_type='continuous')

    # Model states
    x = model.set_variable(var_type='states', var_name='x', shape = (4, 1))
    dx = model.set_variable(var_type='states', var_name='dx', shape = (4, 1))

    # Model inputs
    u = model.set_variable(var_type='inputs', var_name='u', shape = (2, 1))

    # State measurements
    x_meas = model.set_meas('x_meas', x, meas_noise = True)

    # Input measurements
    u_meas = model.set_meas('u_meas', u, meas_noise = False)


    # Model parameters
    k_m0_L = model.set_variable('parameter', 'k_m0_L')
    k_m0_T = model.set_variable('parameter', 'k_m0_T')
    k_m_L = model.set_variable('parameter', 'k_m_L')
    k_m_T = model.set_variable('parameter', 'k_m_T')
    k_p_L = model.set_variable('parameter', 'k_p_L')
    k_p_T = model.set_variable('parameter', 'k_p_T')
    g_m_L = model.set_variable('parameter', 'g_m_L')
    g_m_T = model.set_variable('parameter', 'g_m_T')
    g_p_L = model.set_variable('parameter', 'g_p_L')
    g_p_T = model.set_variable('parameter', 'g_p_T')
    theta_LacI = model.set_variable('parameter', 'theta_LacI')
    theta_TetR = model.set_variable('parameter', 'theta_TetR')
    theta_IPTG = model.set_variable('parameter', 'theta_IPTG')
    theta_aTc = model.set_variable('parameter', 'theta_aTc')
    eta_LacI = 2.00
    eta_TetR = 2.00
    eta_IPTG = 2.00
    eta_aTc = 2.00

    model.set_rhs('x', dx)
    
    dx_next = vertcat(
        k_m0_L + k_m_L*(1 / (1 + ((x[3]/theta_TetR) * (1 / (1 + (u[0]/theta_aTc)**eta_aTc)))**eta_TetR)) - g_m_L * x[0],
        k_m0_T + k_m_T*(1 / (1 + ((x[2]/theta_LacI) * (1 / (1 + (u[1]/theta_IPTG)**eta_IPTG)))**eta_LacI)) - g_m_T * x[1],
        k_p_L * x[0] - g_p_L * x[2],
        k_p_T * x[1] - g_p_T * x[3]
    )
    
    model.set_rhs('dx', dx_next, process_noise = False)
    
    model.setup()

    return model


# MHE Estimator

This function configure and setup the MHE estimator, given the model previously defined.

Going into detail, it defines the objective function for the MHE problem and the weighting matrices associated. Furthermore, define constraints on parameters values.

In [3]:
def template_mhe(model, setup_mhe):

    mhe = do_mpc.estimator.MHE(model, ['k_m0_L', 'k_m0_T', 'k_m_L', 'k_m_T', 'k_p_L', 'k_p_T', 'g_m_L', 'g_m_T', 'g_p_L', 'g_p_T', 'theta_LacI',
                               'theta_TetR', 'theta_IPTG', 'theta_aTc'])

    mhe.set_param(**setup_mhe)
    
    Px = 50*np.eye(8)
    Pv = 10*np.diag(np.array([1,1,1,1]))
    Pp = 1000*np.eye(14)
    mhe.set_default_objective(Px, Pv, Pp)

    mhe.bounds['lower','_p_est', 'k_m0_L'] = 0
    mhe.bounds['lower','_p_est', 'k_m0_T'] = 0
    mhe.bounds['lower','_p_est', 'k_m_L'] = 0
    mhe.bounds['lower','_p_est', 'k_m_T'] = 0
    mhe.bounds['lower','_p_est', 'k_p_L'] = 0
    mhe.bounds['lower','_p_est', 'k_p_T'] = 0
    mhe.bounds['lower','_p_est', 'g_m_L'] = 0
    mhe.bounds['lower','_p_est', 'g_m_T'] = 0
    mhe.bounds['lower','_p_est', 'g_p_L'] = 0
    mhe.bounds['lower','_p_est', 'g_p_T'] = 0
    mhe.bounds['lower','_p_est', 'theta_LacI'] = 0
    mhe.bounds['lower','_p_est', 'theta_TetR'] = 0
    mhe.bounds['lower','_p_est', 'theta_IPTG'] = 0
    mhe.bounds['lower','_p_est', 'theta_aTc'] = 0

    mhe.setup()

    return mhe


# Simulator

This function configure and setup the simulator.h.

In [4]:
def template_simulator(model, t_step):

    simulator = do_mpc.simulator.Simulator(model)
    simulator.set_param(t_step=t_step)

    p_template_sim = simulator.get_p_template()

    def p_fun_sim(t_now):

        p_template_sim['k_m0_L'] = 3.20e-2
        p_template_sim['k_m0_T']= 1.19e-1
        p_template_sim['k_m_L'] = 8.30
        p_template_sim['k_m_T'] = 2.06
        p_template_sim['k_p_L'] = 9.726e-1
        p_template_sim['k_p_T'] = 9.726e-1
        p_template_sim['g_m_L'] = 1.386e-1
        p_template_sim['g_m_T'] = 1.386e-1
        p_template_sim['g_p_L'] = 1.65e-2
        p_template_sim['g_p_T'] = 1.65e-2
        p_template_sim['theta_LacI'] = 31.94
        p_template_sim['theta_TetR'] = 30.00
        p_template_sim['theta_IPTG'] = 9.06e-2
        p_template_sim['theta_aTc'] = 11.65
        return p_template_sim
        
    simulator.set_p_fun(p_fun_sim)

    simulator.setup()

    return simulator
